In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
# np.set_printoptions(precision=4, threshold=15,suppress=True)
pd.options.display.max_rows = 20

**多项式朴素贝叶斯分类器**  
多项式分布(Multinomial Distribution)是二项式分布的推广。二项式做n次伯努利实验，规定了每次试验的结果只有两个，如果现在还是做n次试验，只不过每次试验的结果可以有多m个，且m个结果发生的概率互斥且和为1，则发生其中一个结果X次的概率就是多项式分布。  

多项式朴素贝叶斯的工作方式类似于高斯朴素贝叶斯，但假设这些特征是多项式分布的。 在实践中，这意味着当我们具有离散数据（例如，电影评级范围为 1 到 5）时，通常使用该分类器。

分布参数由每类 y 的 $\theta_y = (\theta_{y_1},\ldots,\theta_{y_n})$ 向量决定， 式中 n 是特征的数量(对于文本分类，是词汇量的大小) $\theta_{y_i}$ 是样本中属于类 y 中特征 i 概率 $P(x_i \mid y)$ 。

参数 $\theta_y$ 使用平滑过的最大似然估计法来估计，即相对频率计数:

$$\hat{\theta}{y_i} = \frac{ N{y_i} + \alpha}{N_y + \alpha n}$$

式中$N_{y_i} = \sum_{x \in T} x_i$是 训练集T中特征i在类y中出现的次数，$N_{y} = \sum_{i=1}^{|T|} N_{y_i}$ 是类 y 中出现所有特征的计数总和。

先验平滑因子 $\alpha \ge 0$ 为在学习样本中没有出现的特征而设计，以防在将来的计算中出现0概率输出。 把 $\alpha = 1$ 被称为拉普拉斯平滑(Lapalce smoothing)，而 $\alpha < 1$ 被称为Lidstone平滑方法(Lidstone smoothing)

# 朴素贝叶斯的文本分类
---
使用朴素贝叶斯进行文本分类；我们将有一组带有相应类别的文本文档，我们将训练一个朴素贝叶斯算法，来学习预测新的没见过的实例的类别。这项简单的任务有许多实际应用；可能是最知名和广泛使用的**垃圾邮件过滤**。在本节中，我们将尝试使用可以从 scikit-learn 中检索的数据集，对新闻组消息进行分类。该数据集包括来自 20 个不同主题的大约 19,000 条新闻组信息，从政治和宗教到体育和科学。

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfVectorizer  # 文本特征提取
from sklearn.pipeline import Pipeline

fetch_20newsgroups(data_home=None, subset=’train’, categories=None, shuffle=True, random_state=42, remove=(), download_if_missing=True)

- data_home指的是数据集的地址，如果默认的话，所有的数据都会在'~/scikit_learn_data'文件夹下.   
    cd \site-packages\sklearn\datasets打开twenty_newsgroups.py文件 修改`archive_path`
- subset就是train,test,all三种可选，分别对应训练集、测试集和所有样本。 
- categories:是指类别，如果指定类别，就会只提取出目标类，如果是默认，则是提取所有类别出来。 
- shuffle:是否打乱样本顺序，如果是相互独立的话。 
- random_state:打乱顺序的随机种子 
- remove:是一个元组，用来去除一些停用词的，例如标题引用之类的。 
- download_if_missing: 如果数据缺失，是否去下载。

In [ ]:
news = fetch_20newsgroups(subset='all')  #  [‘train’ or ‘test’, ‘all’, optional]

返回的数据bunch  
* bunch.data: list, length [n_samples]
* bunch.target: array, shape [n_samples]
* bunch.filenames: list, length [n_samples]
* bunch.DESCR: a description of the dataset.
* bunch.target_names: a list of categories of

In [ ]:
type(news.data), type(news.target), type(news.target_names)

In [ ]:
news.target_names  # 新闻的类别名称

In [ ]:
news.data[0], news.target_names[news.target[0]]  # 查看第一个实例

## 预处理数据
---
我们的机器学习算法只能用于数字数据，因此我们的下一步是将基于文本的数据集转换为数字数据集。目前我们只有一个特征，即消息的文本内容；我们需要一些函数，将文本转换为一组有意义的数字特征。直观地，我们可以尝试查看每个文本类别中使用的单词（或更确切地说，标记，包括数字或标点符号），并尝试表示每个类别中每个单词的频率分布。

scikit-learn提供了从文本内容中提取数字特征的最常见方法，即：

*   **令牌化（tokenizing）** 对每个可能的词令牌分成字符串并赋予整数形的id，例如通过使用空格和标点符号作为令牌分隔符。
*   **统计（counting）** 每个词令牌在文档中的出现次数。
*   **标准化（normalizing）** 在大多数的文档 / 样本中，可以减少重要的次令牌的出现次数的权重。。

在该方案中，特征和样本定义如下：

*   每个**单独的令牌发生频率**（标准化或不标准化）被视为一个**特征**。
*   给定**文档**中所有的令牌频率向量被看做一个多元sample**样本**。

因此，文本的集合可被表示为矩阵形式，每行对应一条文本，每列对应每个文本中出现的词令牌(如单个词)。

我们称**向量化**是将文本文档集合转换为数字集合特征向量的普通方法。 这种特殊思想（令牌化，计数和归一化）被称为 **Bag of Words** 词袋 或 “Bag of n-grams” 模型。 文档由单词出现来描述，同时完全忽略文档中单词的**相对位置信息**。词袋模型是典型的 high-dimensional sparse datasets（**高维稀疏数据集**） 。 我们可以通过只在内存中保存特征向量中非 0 的部分以节省大量内存。

scipy.sparse 矩阵正是能完成上述操作的数据结构，同时 scikit-learn 有对这样的数据结构的内置支持。


`sklearn.feature_extraction.text`模块具有一些有用的工具，可以从文本文档构建数字特征向量:
`CountVectorizer`，`HashingVectorizer`和`TfidfVectorizer`。它们之间的区别在于它们为获得数字特征而执行的计算。
- `CountVectorizer`基本上从文本语料库中创建单词词典。然后，将每个实例转换为数字特征的向量，其中每个元素将是特定单词在文档中出现的次数的计数。

- `HashingVectorizer`，则是在内存中限制并维护字典，实现了 将标记映射到特征索引的散列函数，然后计算`CountVectorizer`中的计数。

- `TfidfVectorizer`的工作方式与`CountVectorizer`类似，但更高级的计算称为**词语频率逆文档频率（TF-IDF）**。这是用于测量在文档或语料库中单词的重要性的统计量。直观地说，它在当前文档中查找中更频繁的单词，与它们在整个文档集中的频率的比值。您可以将此视为一种方法，标准化结果并避免单词过于频繁而无法用于表征实例。

In [ ]:
# 对简约的文本语料库进行 tokenize（分词）和统计单词出现频数
vectorizer = CountVectorizer()
vectorizer

In [ ]:
corpus = [
     'This is the first document.',
     'This is the second second document.',
     'And the third one.',
     'Is this the first document?',
]

In [ ]:
X = vectorizer.fit_transform(corpus)
X

默认配置通过提取至少 2 个字母的单词来对 string 进行分词。做这一步的函数可以显式地被调用:

In [ ]:
analyze = vectorizer.build_analyzer()
analyze

In [ ]:
analyze("This is a text document to analyze.")

analyzer 在拟合过程中找到的每个 term（项）都会被分配一个唯一的整数索引，对应于 resulting matrix（结果矩阵）中的一列。此列的一些说明可以被检索如下:

In [ ]:
vectorizer.get_feature_names()

In [ ]:
print(X.toarray())  # 每行为一个向量, 每一列代表这个feature是否出现和次数

从 feature 名称到 column index（列索引） 的逆映射存储在 vocabulary_ 属性中:

In [ ]:
vectorizer.vocabulary_

因此，在未来对 transform 方法的调用中，在 training corpus （训练语料库）中没有看到的单词将被完全忽略:

In [ ]:
vectorizer.transform(["what does't kill you makes you stronger"]).toarray()

请注意，在前面的 corpus（语料库）中，第一个和最后一个文档具有完全相同的词，因为被编码成相同的向量。 特别是我们丢失了最后一个文件是一个疑问的形式的信息。为了防止词组顺序颠倒，除了提取一元模型 1-grams（个别词）之外，我们还可以提取 2-grams 的单词:

In [ ]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1)  # \b 匹配空字符串但只在单词开始或结尾的位置

In [ ]:
analyze = bigram_vectorizer.build_analyzer()
analyze('Bi-grams are cool!')  # 除了一元模型, 还有2个词连着取的

由 vectorizer（向量化器）提取的 vocabulary（词汇）因此会变得更大，同时可以在定位模式时消除歧义:



In [ ]:
X_2 = bigram_vectorizer.fit_transform(corpus).toarray()
X_2

In [ ]:
feature_index = bigram_vectorizer.vocabulary_.get('is this')
feature_index

In [ ]:
X_2[:, feature_index]  # 只在最后一行出现

### 从出现次数到出现频率
出现次数的统计是非常好的开始，但是有个问题：长的文本相对于短的文本有更高的单词平均出现次数，尽管他们可能在描述同一个主题。

为了避免这些潜在的差异，只需将各文档中每个单词的出现次数除以该文档中所有单词的总数：这些新的特征称之为词频 tf (Term Frequencies)。

另一个在词频的基础上改良是，降低在该训练文集中的很多文档中均出现的单词的权重，从而突出那些仅在该训练文集中在一小部分文档中出现的单词的信息量。

这种方法称为 tf–idf ，全称为 “Term Frequency times Inverse Document Frequency” 。

tf 和 tf–idf 都可以按照下面的方式计算:

In [ ]:
X.shape

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
X_tfidf  = TfidfTransformer().fit_transform(X)  # 通过tf-idf转换器(transformer), 将向量化（vectorizer）矩阵转为tfidf矩阵
X_tfidf

## 训练分类器
###  构建Pipeline (管道)
为了使得 向量化（vectorizer） => 转换器（transformer） => 分类器（classifier） 过程更加简单,scikit-learn 提供了一个 Pipeline 类，操作起来像一个复合分类器.
> Pipeline: 链式评估器<br></br>
> Pipeline 可以把多个评估器链接成一个。这个是很有用的，因为处理数据的步骤一般都是固定的，例如特征选择、标准化和分类。Pipeline 在这里有多种用途:
> * **便捷性和封装性** 你只要对数据调用 fit和 predict 一次来适配所有的一系列评估器。
> * **联合的参数选择** 你可以一次grid search管道中所有评估器的参数。
> * **安全性** 训练转换器和预测器使用的是相同样本，管道有助于防止来自测试数据的统计数据泄露到交叉验证的训练模型中。<br></br>
> 管道中的所有评估器，除了最后一个评估器，管道的所有评估器必须是转换器。 (例如，必须有 `transform` 方法). 最后一个评估器的类型不限（转换器、分类器等等）

Pipeline 使用一系列 (key, value) 键值对来构建,其中 key 是你给这个步骤起的名字， value 是一个评估器对象:

In [ ]:
HashingVectorizer?

In [ ]:
count_clf = Pipeline([('count', CountVectorizer()), ('clf', MultinomialNB())])
hash_clf = Pipeline([('hash', HashingVectorizer(binary=True)), ('clf', MultinomialNB())])
tfidf_clf = Pipeline([('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])
#  TfidfVectorizer ，它将 CountVectorizer 和 TfidfTransformer 的所有选项组合在一个单例模型中:
count_tfidf_clf = Pipeline([('count', CountVectorizer()), ('transformer', TfidfTransformer()), ('clf', MultinomialNB())]) 

In [ ]:
count_clf.fit(news.data,  news.target)
count_clf.score(news.data, news.target)

In [ ]:
hash_clf.fit(news.data,  news.target)
hash_clf.score(news.data, news.target)

In [ ]:
tfidf_clf.fit(news.data,  news.target)
tfidf_clf.score(news.data, news.target)

## k-fold cross validation(k折交叉验证)
将数据集分为k个大小相同, 互不相交的子集, 每次使用k-1个子集训练模型, 用剩下的子集测试模型. 对k个不同的选择进行测试, 选取最佳模型.
使用交叉验证最简单的方法是在估计器和数据集上调用` cross_val_score `辅助函数。

In [ ]:
from sklearn.model_selection import cross_val_score
def evaluate_cross_validation(clf, X, y, k=5):
    scores = cross_val_score(clf, X, y, cv=k)
    print(scores)
    print(f'mean socre: {np.mean(scores):.3f}')


In [ ]:
evaluate_cross_validation(count_clf, news.data, news.target, k=5)

In [ ]:
evaluate_cross_validation(hash_clf, news.data, news.target, k=5)

In [ ]:
evaluate_cross_validation(tfidf_clf, news.data, news.target, k=5)

`CountVectorizer`和`TfidfVectorizer`具有相似的表现，并且比`HashingVectorizer`好得多。  
继续使用`TfidfVectorizer`, 可以尝试通过尝试将文本文档 解析为具有不同**正则表达式**的标记来改进结果.

默认正则表达式选择2个或更多字母数字字符的标记（标点符号被完全忽略，始终视为标记分隔符）:`r"\b\w\w+\b"`,也许还考虑斜线和点可以改善分词, 如:`wi-Fi`, `site.com`

In [ ]:
clf_regex = Pipeline([
    ('tfidf', TfidfVectorizer(token_pattern=r'\b[a-z0-9_\-\.]+[a-z][a-z0-9_\-\.]+\b')),
    ('clf', MultinomialNB())])

In [ ]:
evaluate_cross_validation(clf_regex, news.data, news.target, k=5)

使用**stop words**
内置的英文停止词列表: sklearn\feature_extraction\stop_words.py 

In [ ]:
# english
clf_stop_words = Pipeline([
    ('tfidf', TfidfVectorizer(token_pattern=r'\b[a-z0-9_\-\.]+[a-z][a-z0-9_\-\.]+\b', stop_words='english')),
    ('clf', MultinomialNB())])

In [ ]:
evaluate_cross_validation(clf_stop_words, news.data, news.target, k=5)

In [ ]:
clf_alpha = Pipeline([
    ('tfidf', TfidfVectorizer(token_pattern=r'\b[a-z0-9_\-\.]+[a-z][a-z0-9_\-\.]+\b', stop_words='english')),
    ('clf', MultinomialNB(alpha=0.01))])  # 调整 alpha  1.0 ->0.01

In [ ]:
evaluate_cross_validation(clf_alpha, news.data, news.target, k=5)

## 评估表现

In [ ]:
from sklearn import metrics
def train_and_evaluate(clf, X_train, X_test, y_train, y_test):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(f'训练集score: {clf.score(X_train, y_train)}')
    print(f'测试集score: {clf.score(X_test, y_test)}')
    # 查看主要的分类报告（包含每个类的精确率和召回率）
    print(metrics.classification_report(y_test, y_pred))
    # 混淆矩阵
    print(metrics.confusion_matrix(y_test, y_pred))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(news.data, news.target, test_size=0.2)
train_and_evaluate(clf_alpha, X_train, X_test, y_train, y_test)

In [ ]:
# 查看向量化器，我们可以看到哪些标记已用于创建我们的字典
len(clf_alpha.named_steps['tfidf'].get_feature_names())

In [ ]:
clf_alpha.named_steps['tfidf'].get_feature_names()